<a href="https://colab.research.google.com/github/knkarthick/bart-large-xsum-samsum-dataset/blob/main/Abstractive_Dialogue_Summarization_BART_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets transformers rouge-score nltk py7zr

     |████████████████████████████████| 225kB 26.4MB/s 
     |████████████████████████████████| 2.3MB 40.8MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 112kB 53.1MB/s 
     |████████████████████████████████| 245kB 53.2MB/s 
     |████████████████████████████████| 3.3MB 44.2MB/s 
     |████████████████████████████████| 901kB 42.7MB/s 
     |████████████████████████████████| 1.9MB 37.0MB/s 
     |████████████████████████████████| 122kB 61.6MB/s 
     |████████████████████████████████| 2.2MB 38.3MB/s 
     |████████████████████████████████| 358kB 58.1MB/s 
ERROR: transformers 4.6.1 has requirement huggingface-hub==0.0.8, but you'll have huggingface-hub 0.0.9 which is incompatible.


# Fine-tuning a model on a summarization task

In [ ]:
model_checkpoint = "facebook/bart-base"

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("samsum")
metric = load_metric("rouge")

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


## Preprocessing the data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    "test-dialogue-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


LookupError: ignored

In [ ]:
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.809900,1.551408,46.661900,23.328500,39.481100,43.048200,17.767700


{'eval_gen_len': 17.7677,
 'eval_loss': 1.551408290863037,
 'eval_rouge1': 46.6619,
 'eval_rouge2': 23.3285,
 'eval_rougeL': 39.4811,
 'eval_rougeLsum': 43.0482}

Don't forget to [upload your model](https://huggingface.co/transformers/model_sharing.html) on the [🤗 Model Hub](https://huggingface.co/models). You can then use it only to generate results like the one shown in the first picture of this notebook!

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  
tokenizer_pre = AutoTokenizer.from_pretrained("lidiya/bart-base-samsum")
model_pre = AutoModelForSeq2SeqLM.from_pretrained("lidiya/bart-base-samsum")

In [ ]:
model_pre

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        

In [ ]:
from transformers import pipeline

dialog_summarizer = pipeline("summarization", model=model_pre, tokenizer=tokenizer_pre)

In [ ]:
text = raw_datasets["test"][100]['dialogue']
text

"Chris: Hi there! Where are you? Any chance of skyping?\r\nRick: Hi! Our last two days in Cancun before flying to Havana. Yeah, skyping is an idea. When would it suit you?\r\nRick: We don't have the best of connections in the room but I can get you pretty well in the lobby.\r\nChris: What's the time in your place now?\r\nRick: 6:45 pm\r\nChris: It's a quarter to one in the morning here. Am still in front of the box.\r\nRick: Gracious me! Sorry mate. You needn't have answered.\r\nChris: 8-D\r\nRick: Just tell me when we could skype.\r\nChris: Preferably in the evening. Just a few hours earlier than now. And not tomorrow.\r\nRick: Shute! Only tomorrow makes sense as there's no workable internet in Cuba.\r\nChris: Could you make it like 3 pm your time?\r\nRick: Sure.\r\nChris: Perfect. So talk to you tomorrow.\r\nChris: Give my love to Helen please.\r\nRick: I will. Thx."

In [ ]:
result = dialog_summarizer(text, min_length=10, max_length=40)

In [ ]:
result[0]['summary_text']

'Rick and Chris are skyping in Cancun before flying to Havana. Chris will talk to Helen tomorrow at 3 pm.'

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.809900,1.551408,46.661900,23.328500,39.481100,43.048200,17.767700
0,1.809900,1.587741,44.993200,21.728600,38.192100,41.267200,17.665400


{'eval_gen_len': 17.6654,
 'eval_loss': 1.5877410173416138,
 'eval_rouge1': 44.9932,
 'eval_rouge2': 21.7286,
 'eval_rougeL': 38.1921,
 'eval_rougeLsum': 41.2672}